In [1]:
import json

In [3]:
import requests
from urllib.parse import urlencode
import uuid
import hashlib
import base64
from email.utils import formatdate

class EDAN:
    edan_base = 'http://edan.si.edu/metadata/v2.0/collections/search.htm' 
    
    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key
        self.session = requests.Session()
    def query(self, query_string, start=0, rows=10):
        request_date = formatdate()
        nonce = str(uuid.uuid4())
        query_params = {'q': query_string,
                        'rows': rows,
                        'start': start}
        query_param_string = urlencode(query_params)
        string_to_sign = '\n'.join([nonce, query_param_string, request_date, 
                                    self.app_key])

        hashed_string = hashlib.sha1(string_to_sign.encode()).hexdigest()
        encoded_string = base64.b64encode(hashed_string.encode())
        request_headers = {'X-AppId': self.app_id,
                           'X-Nonce': nonce,
                           'X-RequestDate': request_date,
                           'X-AuthContent': encoded_string}

        r = self.session.get(self.edan_base, params = query_params, 
                             headers = request_headers)
        response_json = r.json()
        return response_json
    
    def fetch_all(self, query_string, step=100):
        results = []
        quick_result = self.query(query_string, rows=1)
        if 'rowCount' in quick_result:
            result_count = quick_result['rowCount']
        if result_count > 0:
            for start in range(0, result_count, step):
                edan_json = self.query(query_string, start=start, rows=step)
                edan_rows = edan_json['rows']
                results += edan_rows
        return results

In [4]:
with open('edan_api_key.json') as api_json:
    edan_key = json.load(api_json)
print(edan_key['APP_ID'])

OCIO_DATA_SCIENCE


In [5]:
edan = EDAN(edan_key['APP_ID'], edan_key['APP_KEY'])

In [6]:
test_call = edan.query("02497538", start=10, rows=1)
test_call

{'rows': [],
 'facetQueries': {},
 'facets': {'culture': [],
  'date': [['1940s', 1]],
  'object_type': [],
  'online_media_type': [['Images', 1]],
  'online_visual_material': [['true', 1]],
  'place': [['Brazil', 1],
   ['Minas Gerais', 1],
   ['South America - Neotropics', 1]],
  'set_name': [['Botany', 1], ['Flowering plants and ferns', 1]],
  'topic': [['Dicotyledonae', 1]],
  'type': [['edanmdm', 1]],
  'unit_code': [['NMNHBOTANY', 1]],
  'p.edanmdm.indexedstructured.common_name': [['jagua', 1]],
  'p.edanmdm.indexedstructured.exhibition_building': [],
  'p.edanmdm.indexedstructured.name': [['Barreto, H. L.', 1]],
  'p.edanmdm.descriptivenonrepeating.data_source': [['NMNH - Botany Dept.',
    1]],
  'p.edanmdm.indexedstructured.geo_age-era': [],
  'p.edanmdm.indexedstructured.geo_age-series': [],
  'p.edanmdm.indexedstructured.geo_age-stage': [],
  'p.edanmdm.indexedstructured.geo_age-system': [],
  'p.edanmdm.indexedstructured.language': [],
  'p.edanmdm.indexedstructured.occupat

In [7]:
results = edan.fetch_all("02497538")
len(results)

1

In [10]:
print(json.dumps(results[0], indent=2))

{
  "id": "edanmdm-nmnhbotany_13711420",
  "title": "Genipa americana L.",
  "unitCode": "NMNHBOTANY",
  "linkedId": "0",
  "type": "edanmdm",
  "url": "edanmdm:nmnhbotany_13711420",
  "content": {
    "descriptiveNonRepeating": {
      "record_ID": "nmnhbotany_13711420",
      "online_media": {
        "mediaCount": 1,
        "media": [
          {
            "thumbnail": "https://ids.si.edu/ids/deliveryService/id/ark:/65665/m3cc16fb8b35924347b0005d899c27813f/90",
            "idsId": "ark:/65665/m3cc16fb8b35924347b0005d899c27813f",
            "usage": {
              "access": "Usage conditions apply"
            },
            "guid": "http://n2t.net/ark:/65665/m3cc16fb8b-3592-4347-b000-5d899c27813f",
            "type": "Images",
            "content": "https://ids.si.edu/ids/deliveryService/id/ark:/65665/m3cc16fb8b35924347b0005d899c27813f"
          }
        ]
      },
      "guid": "http://n2t.net/ark:/65665/35855344c-24b6-4b10-be26-27d0a788f485",
      "title_sort": "GENIPA 

In [15]:
edan_base = 'http://edan.si.edu/metadata/v2.0/collections/getFacets.htm' 

start=0
rows=10
request_date = formatdate()
nonce = str(uuid.uuid4())
query_params = {'q': "02497538",
                'rows': rows,
                'start': start}
query_param_string = urlencode(query_params)
string_to_sign = '\n'.join([nonce, query_param_string, request_date, 
                             edan_key['APP_KEY']])

hashed_string = hashlib.sha1(string_to_sign.encode()).hexdigest()
encoded_string = base64.b64encode(hashed_string.encode())
request_headers = {'X-AppId': edan_key['APP_ID'],
                   'X-Nonce': nonce,
                   'X-RequestDate': request_date,
                   'X-AuthContent': encoded_string}

r = requests.get(edan_base, params = query_params, 
                     headers = request_headers)
response_json = r.json()
print(response_json)

{'facets': {'culture': [], 'data_source': [['NMNH - Botany Dept.', 1]], 'date': [['1940s', 1]], 'object_type': [], 'online_media_type': [['Images', 1]], 'online_visual_material': [['true', 1]], 'place': [['Brazil', 1], ['Minas Gerais', 1], ['South America - Neotropics', 1]], 'set_name': [['Botany', 1], ['Flowering plants and ferns', 1]], 'topic': [['Dicotyledonae', 1]], 'type': [['edanmdm', 1]], 'metadata_usage': [['CC0', 1]], 'media_usage': [['Usage conditions apply', 1]]}}
